In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/MyDrive/Colab/졸업작품

/content/drive/MyDrive/Colab/졸업작품


In [ ]:
#!git clone https://github.com/JaidedAI/EasyOCR.git

In [ ]:
#!git clone https://github.com/clovaai/deep-text-recognition-benchmark.git

In [ ]:
# !unzip -qq 'New_sample.zip'

# Load Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import json
import random
import shutil

In [ ]:
image_folder_path = '/content/drive/MyDrive/Colab/졸업작품/EasyOCR/custom_dataset/images'
label_folder_path = '/content/drive/MyDrive/Colab/졸업작품/EasyOCR/custom_dataset/labels'

# Prepare dataset for finetuning CRAFT

In [ ]:
# change craft label form(json to txt)
# annotation.bbox[x, y, width, height]
def make_txt_label(label_data, img_name, train = True):
  '''
  label_data : json form label(bbox, text)
  img_name : image name(example.png, example.jpg)
  train: make txt label for train model(if train == True) // make txt label for evaluate model(if train == False)
  '''
  label_text = ''
  for i, annot in enumerate(label_data['annotations']):
      bbox = label_data['annotations'][i]['annotation.bbox']
      label_text += str(bbox[0])+','+str(bbox[1])+','
      label_text += str(bbox[0]+bbox[2])+','+str(bbox[1])+','
      label_text += str(bbox[0]+bbox[2])+','+str(bbox[1]+bbox[3])+','
      label_text += str(bbox[0])+','+str(bbox[1]+bbox[3])+','
      label_text += label_data['annotations'][i]['annotation.text']+'\n'

  if train:
    File = open("/content/drive/MyDrive/Colab/졸업작품/EasyOCR/trainer/craft/data_root_dir/ch4_training_localization_transcription_gt/gt_" + img_name[:-4] +'.txt', "w")
    shutil.move(image_folder_path +'/' +img_name,"/content/drive/MyDrive/Colab/졸업작품/EasyOCR/trainer/craft/data_root_dir/ch4_training_images")
  else:
    File = open("/content/drive/MyDrive/Colab/졸업작품/EasyOCR/trainer/craft/data_root_dir/ch4_test_localization_transcription_gt/gt_" + img_name[:-4] +'.txt', "w")
    shutil.move(image_folder_path + '/' +img_name,"/content/drive/MyDrive/Colab/졸업작품/EasyOCR/trainer/craft/data_root_dir/ch4_test_images")
  File.write(label_text)
  File.close()

In [ ]:
def prepare_custom_craft_dataset(train_size = 0.8, image_list = None):
  '''
  train_size : train size ratio(float 0~1)
  image_list: image name list
  '''
  for i, img_name in enumerate(image_list):
      data_size = len(image_list)
      label_name = img_name[:-4]
      label_path = label_folder_path + '/' + label_name+'.json'
      with open (label_path, 'r') as f:
        label_data = json.load(f)

      # for train dataset
      if i < data_size * train_size:
        make_txt_label(label_data = label_data, img_name = img_name, train = True)
      # for test dataset
      else:
        make_txt_label(label_data = label_data, img_name = img_name, train = False)

In [ ]:
random.seed(42)
image_list = os.listdir(image_folder_path)
random.shuffle(image_list)
prepare_custom_craft_dataset(train_size = 0.8, image_list = image_list)

In [ ]:
# image_data = []
# label_data = []

# for name in os.listdir(image_folder_path)[:10]:
#   path = image_folder_path + '/' + name
#   # read image with grayscale
#   img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
#   img = np.asarray(img)
#   image_data.append(img)

# image_data = np.asarray(image_data)

# for name in os.listdir(label_folder_path)[:10]:
#   path = label_folder_path + '/' + name
#   with open (path, 'r') as f:
#     label = json.load(f)
#   label_data.append(label)

# label_data = np.asarray(label_data)

# print(image_data.shape)
# print(label_data.shape)

In [ ]:
# # 영상 이진화(임계값 설정 해보면서 결과보기)
# ret, img1 = cv2.threshold(img1,0, 255, cv2.THRESH_BINARY)
# plt.imshow(img1, cmap = 'gray');
# print(np.unique(img1))

# # 잡음 제거
# img1_filtered = cv2.bilateralFilter(img1)

# CRAFT

In [ ]:
%cd /content/drive/MyDrive/Colab/졸업작품/EasyOCR/trainer/craft

/content/drive/MyDrive/Colab/졸업작품/EasyOCR/trainer/craft


In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 26.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.9 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=c67085e594c1d66a50f98ee713cc4a292d4a16d03bb48c75484fd11f18af2ab7
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
!python train.py --yaml=custom_data_train

-------------------- Options --------------------
data_root_dir: ./data_root_dir/
mode: weak_supervision
results_dir: ./exp/
score_gt_dir: None
test:
  custom_data:
    canvas_size: 2240
    cuda: true
    link_threshold: 0.2
    low_text: 0.5
    mag_ratio: 1.75
    poly: false
    test_data_dir: ./data_root_dir/
    test_set_size: 500
    text_threshold: 0.75
    vis_opt: false
  trained_model: null
train:
  amp: true
  backbone: vgg
  batch_size: 5
  ckpt_path: ./pretrained_model/CRAFT_clr_amp_29500.pth
  data:
    custom_aug:
      random_colorjitter:
        brightness: 0.2
        contrast: 0.2
        hue: 0.2
        option: true
        saturation: 0.2
      random_crop:
        option: true
        ratio:
        - 0.75
        - 1.33
        rnd_threshold: 1.0
        scale:
        - 0.03
        - 0.4
        version: random_resize_crop
      random_horizontal_flip:
        option: true
      random_rotate:
        max_angle: 20
        option: true
      random_scale:
   

# craft weight load error check(나중에 지우기)

In [ ]:
import torch
a = torch.load('/content/drive/MyDrive/Colab/졸업작품/EasyOCR/trainer/craft/pretrained_model/CRAFT_clr_amp_14000.pth', map_location = torch.device('cpu'))
a['craft'].keys()

odict_keys(['module.basenet.slice1.0.weight', 'module.basenet.slice1.0.bias', 'module.basenet.slice1.1.weight', 'module.basenet.slice1.1.bias', 'module.basenet.slice1.1.running_mean', 'module.basenet.slice1.1.running_var', 'module.basenet.slice1.1.num_batches_tracked', 'module.basenet.slice1.3.weight', 'module.basenet.slice1.3.bias', 'module.basenet.slice1.4.weight', 'module.basenet.slice1.4.bias', 'module.basenet.slice1.4.running_mean', 'module.basenet.slice1.4.running_var', 'module.basenet.slice1.4.num_batches_tracked', 'module.basenet.slice1.7.weight', 'module.basenet.slice1.7.bias', 'module.basenet.slice1.8.weight', 'module.basenet.slice1.8.bias', 'module.basenet.slice1.8.running_mean', 'module.basenet.slice1.8.running_var', 'module.basenet.slice1.8.num_batches_tracked', 'module.basenet.slice1.10.weight', 'module.basenet.slice1.10.bias', 'module.basenet.slice1.11.weight', 'module.basenet.slice1.11.bias', 'module.basenet.slice1.11.running_mean', 'module.basenet.slice1.11.running_var

In [ ]:
a.keys()

dict_keys(['iter', 'craft', 'optimizer', 'scaler'])

In [ ]:
craft_state_dict = {}
for key in a['craft'].keys():
  craft_state_dict[key[7:]] = a['craft'][key]

In [ ]:
craft_state_dict.keys()

dict_keys(['basenet.slice1.0.weight', 'basenet.slice1.0.bias', 'basenet.slice1.1.weight', 'basenet.slice1.1.bias', 'basenet.slice1.1.running_mean', 'basenet.slice1.1.running_var', 'basenet.slice1.1.num_batches_tracked', 'basenet.slice1.3.weight', 'basenet.slice1.3.bias', 'basenet.slice1.4.weight', 'basenet.slice1.4.bias', 'basenet.slice1.4.running_mean', 'basenet.slice1.4.running_var', 'basenet.slice1.4.num_batches_tracked', 'basenet.slice1.7.weight', 'basenet.slice1.7.bias', 'basenet.slice1.8.weight', 'basenet.slice1.8.bias', 'basenet.slice1.8.running_mean', 'basenet.slice1.8.running_var', 'basenet.slice1.8.num_batches_tracked', 'basenet.slice1.10.weight', 'basenet.slice1.10.bias', 'basenet.slice1.11.weight', 'basenet.slice1.11.bias', 'basenet.slice1.11.running_mean', 'basenet.slice1.11.running_var', 'basenet.slice1.11.num_batches_tracked', 'basenet.slice2.14.weight', 'basenet.slice2.14.bias', 'basenet.slice2.15.weight', 'basenet.slice2.15.bias', 'basenet.slice2.15.running_mean', 'bas

In [ ]:
# # use library
# langs = ['ko', 'en']

# print("[INFO] OCR'ing input image...")
# reader = Reader(lang_list=langs, gpu=True)
# results = reader.readtext(image_data[1])

[INFO] OCR'ing input image...


In [ ]:
def putText(cv_img, text, x, y, color=(0, 0, 0), font_size=22):
  # Colab이 아닌 Local에서 수행 시에는 gulim.ttc 를 사용하면 됩니다.
  # font = ImageFont.truetype("fonts/gulim.ttc", font_size)
  font = ImageFont.truetype('/usr/share/fonts/truetype/nanum/NanumGothicBold.ttf', font_size)
  img = Image.fromarray(cv_img)

  draw = ImageDraw.Draw(img)
  draw.text((x, y), text, font=font, fill=color)

  cv_img = np.array(img)

  return cv_img

In [ ]:
# loop over the results
for (bbox, text, prob) in results:
  print("[INFO] {:.4f}: {}".format(prob, text))

  (tl, tr, br, bl) = bbox
  tl = (int(tl[0]), int(tl[1]))
  tr = (int(tr[0]), int(tr[1]))
  br = (int(br[0]), int(br[1]))
  bl = (int(bl[0]), int(bl[1]))

	# 추출한 영역에 사각형을 그리고 인식한 글자를 표기합니다.
  cv2.rectangle(image_data[0], tl, br, (0, 255, 0), 2)
  output_image = putText(image_data[0], text, tl[0], tl[1] - 60, (0, 255, 0), 50)
	#cv2.putText(image_data[0], text, (tl[0], tl[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)


# show the output image
plt.imshow("Image", output_image, figsize=(16,10))

TabError: ignored